In [3]:
from keras.layers import Input, Dense
from keras.models import Model
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from numpy import linalg as LA
import sklearn
from sklearn import metrics
import csv
from sklearn import preprocessing

In [5]:
usr_dir="Raw_data/Amp/"
for user in range(1,23,1):
    usr_file=usr_dir+"U"+str(user)+"_LIHF_itd_file.csv"
    print(usr_file)
    #### load values ########
    dataset=pd.read_csv(usr_file,header=None)
    #dataframe=dataset.values
    array=dataset.values
    
    
    ###load data into training set ########
    row_no=array.shape[0]
    col_no=array.shape[1]

    print(row_no)
    print(col_no)
    # normalize the data attributes
    dataframe = preprocessing.normalize(array[:,0:col_no-1])
    
    X_train=dataframe[:,0:col_no-1]
    #X_train=dataframe[0:row_no,0:col_no-1]
    print(X_train.shape)
    
    happy_test=[]
    happy_label=[]
    sad_test=[]
    sad_label=[]
    stressed_test=[]
    stressed_label=[]
    relaxed_test=[]
    relaxed_label=[]
    
    for i in range(row_no):
        if(array[i,col_no-1]==2):
            happy_test.append(list(dataframe[i,0:col_no-1]))
            happy_label.append([2])
        if(array[i,col_no-1]==-2):
            sad_test.append(list(dataframe[i,0:col_no-1]))
            sad_label.append([-2])
        if(array[i,col_no-1]==1):
            stressed_test.append(list(dataframe[i,0:col_no-1])) 
            stressed_label.append([1])
        if(array[i,col_no-1]==0):
            relaxed_test.append(list(dataframe[i,0:col_no-1])) 
            relaxed_label.append([0])
            
    happy_test=np.asarray(happy_test)
    sad_test=np.asarray(sad_test)
    stressed_test=np.asarray(stressed_test)
    relaxed_test=np.asarray(relaxed_test)
    ########## build model #####################
    # this is the input size, size of each row. last column is discarded as it represents label
    INPUT_SIZE=col_no-1
    # this is(16) the size of our encoded representations
    ENCODING_SIZE =10
    # this is our input placeholder
    input_row = Input(shape=(INPUT_SIZE,))
    # "encoded" is the encoded representation of input
    #encoded=Dense(100,activation='relu')(input_row)
    #encoded=Dense(80,activation='relu')(encoded)
    #encoded=Dense(50,activation='relu')(encoded)
    #encoded = Dense(ENCODING_SIZE, activation='relu')(encoded)
    encoded = Dense(ENCODING_SIZE, activation='relu')(input_row)
    
    
    # "decoded" is the lossy reconstruction of input
    #decoded = Dense(50, activation='relu')(encoded)
    #decoded=Dense(80,activation='relu')(decoded)
    #decoded=Dense(100,activation='relu')(decoded)
    #decoded = Dense(INPUT_SIZE, activation='relu')(decoded)
    decoded = Dense(INPUT_SIZE, activation='relu')(encoded)
    # This model maps an input to its reconstruction
    autoencoder = Model(input_row, decoded)
    # this model maps an input to its encoded representation
    encoder = Model(input_row, encoded)
    # create a placeholder for an encoded (16-dimensional) input
    encoded_input = Input(shape=(ENCODING_SIZE,))
    
    # retrieve the last layer of the autoencoder model
    decoder_layer = autoencoder.layers[-1]
    
    # create the decoder model
    
    decoder = Model(encoded_input, decoder_layer(encoded_input))
    
    
    ####################################################################
    
    # configure our model to use mean_squared_error loss function and adam optimizer function
    autoencoder.compile(optimizer='adam', loss='mean_squared_error')

    autoencoder_train=autoencoder.fit(X_train, X_train, epochs=50, batch_size=10,verbose=0, shuffle=True, validation_split=0.3)
    
    ##################### encoded vector for each emotion #################
    similar_vector=np.zeros((4,4))
    encoded_dir="encoded_input/Amp_ISS/"
    raw_file=encoded_dir+"U"+str(user)+"_LIHF_itd_file.csv"
    with open(raw_file,"w") as resultfile:
        writer=csv.writer(resultfile)
        if(len(happy_test)>0):
            print("happy present")
            happy_encoded = encoder.predict(happy_test)
            happy_label=np.array(happy_label)
            happy_array=np.append(happy_encoded,happy_label,axis=1)
            writer.writerows(happy_array)
            #print(type(happy_encoded))
        if(len(sad_test)>0):
            
            print("sad present")
            sad_encoded=encoder.predict(sad_test)
            sad_label=np.array(sad_label)
            sad_array=np.append(sad_encoded,sad_label,axis=1)
            writer.writerows(sad_array)
            sad_sad=metrics.pairwise.cosine_similarity(sad_encoded,sad_encoded)
            similar_vector[1][1]=np.mean(sad_sad)
            
        if(len(stressed_test)>0):
            
            print("stressed present")
            stressed_encoded=encoder.predict(stressed_test)
            stressed_label=np.array(stressed_label)
            stressed_array=np.append(stressed_encoded,stressed_label,axis=1)
            writer.writerows(stressed_array)
            
            stress_stress=metrics.pairwise.cosine_similarity(stressed_encoded,stressed_encoded)
            similar_vector[2][2]=np.mean(stress_stress)
            
        if(len(relaxed_test)>0):
            
            print("relaxed present")
            relaxed_encoded=encoder.predict(relaxed_test)
            relaxed_label=np.array(relaxed_label)
            relaxed_array=np.append(relaxed_encoded,relaxed_label,axis=1)
            writer.writerows(relaxed_array)
            relax_relax=metrics.pairwise.cosine_similarity(relaxed_encoded,relaxed_encoded)
            similar_vector[3][3]=np.mean(relax_relax)
            
    ###################################################################
    
    if(len(happy_test)>0):
        happy_happy=metrics.pairwise.cosine_similarity(happy_encoded,happy_encoded)
        similar_vector[0][0]=np.mean(happy_happy)
        if(len(sad_test)>0):
            happy_sad=metrics.pairwise.cosine_similarity(happy_encoded,sad_encoded)
            similar_vector[0][1]=np.mean(happy_sad) 
        if(len(stressed_test)>0):
                happy_stressed=metrics.pairwise.cosine_similarity(happy_encoded,stressed_encoded)
                similar_vector[0][2]=np.mean(happy_stressed)
        if(len(relaxed_test)>0):
                happy_relaxed=metrics.pairwise.cosine_similarity(happy_encoded,relaxed_encoded)
                similar_vector[0][3]=np.mean(happy_relaxed)
    if(len(sad_test)>0):
        if(len(stressed_test)>0):
            sad_stressed=metrics.pairwise.cosine_similarity(sad_encoded,stressed_encoded)
            similar_vector[1][2]=np.mean(sad_stressed)
        if(len(relaxed_test)>0):
            sad_relaxed=metrics.pairwise.cosine_similarity(sad_encoded,relaxed_encoded)
            similar_vector[1][3]=np.mean(sad_relaxed)
            
    if(len(stressed_test)>0):
            if(len(relaxed_test)>0):
                stress_relaxed=metrics.pairwise.cosine_similarity(stressed_encoded,relaxed_encoded)
                similar_vector[2][3]=np.mean(stress_relaxed)
        
    print(similar_vector)

Raw_data/Amp/U1_LIHF_itd_file.csv
114
2540
(114, 2539)
happy present
sad present
stressed present
relaxed present
[[0.8061983  0.80987877 0.80863726 0.82880366]
 [0.         0.86284792 0.84141564 0.84548825]
 [0.         0.         0.83058918 0.84288424]
 [0.         0.         0.         0.86984533]]
Raw_data/Amp/U2_LIHF_itd_file.csv
45
2540
(45, 2539)
happy present
sad present
stressed present
relaxed present
[[0.73994768 0.73262149 0.77621216 0.75816625]
 [0.         0.99712396 0.94042921 0.8549577 ]
 [0.         0.         0.94549167 0.87127227]
 [0.         0.         0.         0.82144564]]
Raw_data/Amp/U3_LIHF_itd_file.csv
45
340
(45, 339)
happy present
sad present
stressed present
relaxed present
[[0.98541862 0.97944421 0.85298741 0.95072562]
 [0.         0.98183566 0.83823591 0.94389844]
 [0.         0.         0.84649175 0.85432899]
 [0.         0.         0.         0.92680168]]
Raw_data/Amp/U4_LIHF_itd_file.csv
118
1976
(118, 1975)
happy present
sad present
stressed present